In [19]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter as cnt
import string
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.metrics.pairwise import linear_kernel
from preprocess import preprocessing
import csv

In [12]:
all_dev=((pd.read_csv('data/devSet.csv')).dropna())[0:1000]
all_dev.shape

(1000, 6)

In [13]:
train=all_dev.sample(frac=0.8, replace=False, random_state=1)

In [14]:
dev=all_dev[~all_dev.isin(train)].dropna()

In [22]:
train.shape, dev.shape

((800, 6), (200, 6))

In [41]:
def create_model(train=''):
    l_q1=list(train.question1)
    l_q2=list(train.question2)
    l_words=l_q1+l_q2

    # Preprocess text
    pre = preprocessing()
    prep_words=[pre.the_works(text=q) for q in l_words]
    
    # Join tokens per question (That's how the 
    # td-idf vectorizer works)
    joined_words=[" ".join(q) for q in prep_words]
    # Buld Model]
    
    # create word counts first and TF
    cv=CountVectorizer(lowercase=False)
    # this steps generates word counts for the words
    word_count_vector=cv.fit_transform(joined_words)
    
    # Now let's generate IDF values
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    
    # Visualize in dataframe the vector
    #df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
    # sort ascending
    #return df_idf.sort_values(by=['idf_weights'])
    
    # Compute TF-IDF
    # count matrix
    count_vector=cv.transform(joined_words)
 
    # tf-idf scores
    tf_idf_vector=tfidf_transformer.transform(count_vector)
    
    return cv, tfidf_transformer

    #tfidf = TfidfVectorizer(lowercase=False)
    #tfs = tfidf.fit_transform(joined_words)
    #return tfidf, tfs

In [17]:
def evaluate(threshold=0.0, test=''):
    global results_test
    tl_q1=list(test.question1)
    tl_q2=list(test.question2)
    dup = list(test.is_duplicate)
    pre = preprocessing()
    correct=0
    total=0
    for l_q1, l_q2,is_dup in zip(tl_q1, tl_q2, dup):
        
        # Preprocess text
        prep_q1=pre.the_works(text=l_q1)
        prep_q2=pre.the_works(text=l_q2)
        
        # Validation, after removal of
        # stop words, we can get into a situation
        # of having an empty string
        if prep_q1 is np.nan: continue
        if prep_q2 is np.nan: continue
        if len(prep_q1) == 0: continue
        if len(prep_q2) == 0: continue
            
        total=total+1
        # Join tokens per question (That's how the 
        # td-idf vectorizer works)
        #joined_q1=[" ".join(q) for q in prep_q1]
        #joined_q2=[" ".join(q) for q in prep_q2]
    
        q1_vector= tfidf.transform(prep_q1)
        q2_vector= tfidf.transform(prep_q2)
        cosine_similarities = linear_kernel(q1_vector, q2_vector).flatten()
        print(cosine_similarities[0])
        if cosine_similarities[0] > threshold:
            #results_test.append((id, 1))
            print('Correct')
            correct=correct+1
        #else:
            #results_test.append((id, 0))
    return correct/total, threshold

In [56]:
cv, tfidf_transformer=create_model(train=train)

Example to visualize vector

Now let's test

In [69]:
test_text=['This is a complex text','the exam is complex in its text']
count_vector=cv.transform(test_text)
# tf-idf scores
final_vector=tfidf_transformer.transform(count_vector)

In [70]:
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=final_vector[0]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)[0:4]

,tfidf
complex,0.709018
text,0.607663
is,0.357826
0frac1xfracxx1math,0.000000


In [71]:
q1_vector=final_vector[0]
q2_vector=final_vector[1]
linear_kernel(q1_vector, q2_vector).flatten()

array([0.7854759])

In [47]:
first_document_vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [113]:
samples = np.linspace(50, 100, num=20)
eval_thres=[]
for t in samples:
    print(t)
    results_test = []
    prec, thres =evaluate(threshold=t, test=dev)
    eval_thres.append([prec, thres])

50.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50

<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <c

<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 50.0 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 50.0 <c

<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class

<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 52.631578947368425 <class 'numpy.float64'> 0.0
<class

...

<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float6

<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 94.73684210526316 <class 'numpy.float64'> 1.0
<class 'numpy.float6

<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float6

<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 0.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float64'> 97.36842105263159 <class 'numpy.float64'> 1.0
<class 'numpy.float6

KeyboardInterrupt: 

In [110]:
eval_thres

[[0.0, 50.0],
 [0.0, 52.631578947368425],
 [0.0, 55.26315789473684],
 [0.0, 57.89473684210526],
 [0.0, 60.526315789473685],
 [0.0, 63.15789473684211],
 [0.0, 65.78947368421052],
 [0.0, 68.42105263157895],
 [0.0, 71.05263157894737],
 [0.0, 73.6842105263158],
 [0.0, 76.31578947368422],
 [0.0, 78.94736842105263],
 [0.0, 81.57894736842105],
 [0.0, 84.21052631578948],
 [0.0, 86.84210526315789],
 [0.0, 89.47368421052632],
 [0.0, 92.10526315789474],
 [0.0, 94.73684210526316],
 [0.0, 97.36842105263159],
 [0.0, 100.0]]